In [1]:
import pandas as pd

In [2]:
# biogrid data, physical interactions
# entrez id to ensembl id
edge_list = pd.read_csv("../../data/BIOGRID/BIOGRID-ORGANISM-Homo_sapiens-4.4.218.tab3.txt", sep="\t", low_memory=False)
ensembl2entrez = pd.read_csv("../../data/Ensembl/Ensembl2Entrez.txt", sep="\t", header=None).dropna().copy()
ensembl2entrez[1] = ensembl2entrez[1].astype(int)
entrez2ensembl_dict = ensembl2entrez.set_index(1)[0].to_dict()
edge_list = edge_list[edge_list['Entrez Gene Interactor A'].apply(lambda x: str(x).isdigit())]
edge_list = edge_list[edge_list['Entrez Gene Interactor B'].apply(lambda x: str(x).isdigit())]
edge_list[0] = edge_list['Entrez Gene Interactor A'].astype(int).map(entrez2ensembl_dict)
edge_list[1] = edge_list['Entrez Gene Interactor B'].astype(int).map(entrez2ensembl_dict)
edge_list = edge_list[edge_list['Experimental System Type'] == "physical"].copy()
edge_list = edge_list.loc[:, [0, 1]].dropna()
edge_list = edge_list[edge_list[0] != edge_list[1]].copy() # remove self loop

In [3]:
# valid genes
genes = pd.read_csv("../../result/network_GTEx_phastcons/valid_genes", sep="\t")
id2genes = genes.set_index("ID")["genes"].to_dict()
genes2id = genes.set_index("genes")["ID"].to_dict()
genes = genes['genes'].values

In [4]:
# keep valid genes
edge_list = edge_list[edge_list[0].isin(genes)]
edge_list = edge_list[edge_list[1].isin(genes)]

In [5]:
# convert to number ID
edge_list[0] = edge_list[0].map(genes2id)
edge_list[1] = edge_list[1].map(genes2id)

In [6]:
# uniq & sort
edge_list = edge_list.drop_duplicates().sort_values([0, 1])

In [8]:
edge_list.to_csv("../../result/network_GTEx_phastcons/BIOGRID.tsv", sep="\t", header=False, index=False)